<a href="https://colab.research.google.com/github/btlgs2000/dl_intro/blob/master/transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer learning

In [ ]:
from tensorflow import keras

In [ ]:
! git clone https://github.com/btlgs2000/dl_intro

In [ ]:
! wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1vL3uIpBcd-U_xnKD4EKvXGyL7r66CuDJ' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1vL3uIpBcd-U_xnKD4EKvXGyL7r66CuDJ" -O cats_dogs.zip && rm -rf /tmp/cookies.txt

In [ ]:
! unzip /content/cats_dogs.zip

## Caricamento del Dataset

In [ ]:
TRAIN_FOLDER = r'/content/dogs-vs-cats/train'
BATCH_SIZE = 32
IMAGE_SIZE = (96, 96)

In [ ]:
train_ds = keras.preprocessing.image_dataset_from_directory(
    directory=TRAIN_FOLDER,
    labels='inferred',
    label_mode='binary',
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    seed=1,
    validation_split=0.1,
    subset='training')

val_ds = keras.preprocessing.image_dataset_from_directory(
    directory=TRAIN_FOLDER,
    labels='inferred',
    label_mode='binary',
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    seed=1,
    validation_split=0.1,
    subset='validation')

In [ ]:
mobile_net = keras.applications.MobileNetV2(input_shape=(*IMAGE_SIZE, 3), include_top=False, weights='imagenet', pooling='avg')

In [ ]:
mobile_net.summary()

In [ ]:
mobile_net.trainable = False

In [ ]:
inputs = keras.Input(shape=(*IMAGE_SIZE, 3))

In [ ]:
x = keras.applications.mobilenet_v2.preprocess_input(inputs)
x = mobile_net(x, training=False)
outputs = keras.layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)

In [ ]:
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy', metrics=['binary_accuracy'])

In [ ]:
model.fit(x=train_ds, validation_data=val_ds, epochs=1)

In [ ]:
model.save('checkpoint.tf')

In [ ]:
mobile_net.trainable = True

In [ ]:
for layer in model.layers:
    print(layer.trainable)

In [ ]:
model = keras.models.load_model('checkpoint.tf')

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['binary_accuracy'])

In [ ]:
model.fit(x=train_ds, validation_data=val_ds, epochs=1)